In [3]:
import openai
import time
import json
#import transformers
#from transformers import GPT2Tokenizer
import backoff
from openai.error import RateLimitError
import datetime
import pickle
import numpy as np
from numpy.linalg import norm

In [2]:
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [4]:
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

In [5]:
openai.api_key = api_key

In [46]:
""" def gpt3_embedding(content, model='text-similarity-ada-001'):
    try:
        response = openai.Embedding.create(input=content, engine=model)
    except openai.error.APIConnectionError:
        print("Failed") 
    return response['data'][0]['embedding']  """

In [6]:
#API response for any prompt - change settings : temperature to be low, best_of=5, n=3 -> captures completions response
@backoff.on_exception(backoff.expo, RateLimitError)
def response_API(prompt, myKwargs = {}):
    
    kwargs = {"model" :"text-davinci-002",
            "temperature" :0.46,
            "max_tokens": 600,
            "frequency_penalty":1,
            "presence_penalty":0}
    #assign changed values and keep everything else the same
    for kwarg in myKwargs:
        kwargs[kwarg] = myKwargs[kwarg]
        
    try:
        response = openai.Completion.create(prompt=prompt, **kwargs)
    except openai.error.APIConnectionError:
        print("Failed")
    
    return response['choices'][0]['text']

In [7]:
def question_completions_with_backoff(passages): 
       
    question_prompts = ['''Generate one multiple-choice type question about this context. The question should consist of reasoning and procedural steps including all the possiblities of the context. \n
                        The question should be precise and factual, and should address the real world curiosity of the topic.''',
                        
                        '''Generate one objective, to-the-point and firm question about this context. The question should break down the context and specifically pin-point key terminologies and concepts. \n
                        It should be precise, factual and truthful. The question should identify the important terms and accurately summarize them.''' ,
                        
                        '''Generate one thoughtful, creative, steps-based procedural question about this context that starts with Why/How/Where/Who/When. \n
                        It should be detailed, free-flowing, conversational and analytical. The question should spur curiority and inspire discussions and debates.''' ]
    
    n=len(question_prompts)
    questions = []
    qprompts = []
    for p in passages:
        for j in question_prompts:
                #prompt_tokens = calculate_tokens(j)
                #context_tokens = calculate_tokens(p)
                #max_tokens = 300
                
                #while(max_tokens+prompt_tokens+context_tokens < 4096):
                prompt= "%s \n Context : %s" % (j, p)
                #print(prompt)
    
                response = response_API(prompt)
                questions.append(response)
                qprompts.append(j)
                print("Done")
                      
    question_list = [questions[i:i + n] for i in range(0, len(questions), n)]
    
    return question_list, qprompts #for each prompt there is one question generated.

In [8]:
def answer_completions_with_backoff(question_list, passages):
    
    answer_prompt = """ Generate an objective, free flowing, conversational and formal answer about this question, based on the given context. 
    The answer must spur curiosity and enable interactive discussions. It should explain the concept of the context accurately, and be truthful and factual.
    It should be interested and use advanced vocabulary and complex sentence structures. Generate the answer now."""
    
    
    answers = []
    QA_pairs = []
    QA_data_full = []
    
    for index, par in enumerate(passages):
        for i, q in enumerate(question_list[index]):
            
            ans_prompt = "Context : %s \n Question : %s \n Prompt : %s" % (par, q, answer_prompt)
            
            ans_response = response_API(ans_prompt)
            print("Done")
            answers.append(ans_response)
            qa_dict = {"question" : q, "answer" : ans_response}
            
            QA_pairs.append(qa_dict)
            
            data = {}
            data['textbook-paragraph'] = par
            data['GPT-3-Generations'] = {}
            data['GPT-3-Generations']['question'] = q
            data['GPT-3-Generations']['answer'] = ans_response   
            QA_data_full.append(data)
            
    
    return answers, QA_pairs, QA_data_full

start here

In [9]:
#sections data
s = open("/Users/nehasheth/Desktop/Research - AI Chatbot TA/github/data-generator/gpt-3/GPT-3_section_level.json")
sections_data = json.load(s)

""" #full textbook embeddings - vectors
with open("/Users/nehasheth/Desktop/Research - AI Chatbot TA/github/data-generator/prompt_engineering/index.json") as input_file:
    data = json.load(input_file) """
    
s = open("/Users/nehasheth/Desktop/Research - AI Chatbot TA/github/data-generator/split_textbook/paragraphs.json")
paragraphs_data = json.load(s)
paragraphs_list = list(paragraphs_data.values())
    
    

In [10]:
#make a list of only texts from the json file
sections_list = []
for p, item in enumerate(sections_data):
    subtext = item['positive_ctxs']['text']
    sections_list.append(subtext)

In [ ]:
#Generating questions 
question_list, qprompts = question_completions_with_backoff(sections_list[0:2]) 


In [ ]:
answers, QA_pairs, QA_data_full = answer_completions_with_backoff(question_list, sections_list[0:2])

In [ ]:
""" ##OUTPUT FORMAT NEEDED - 
  { 
    "textbook_paragraph": "For some of the....",        
    "GPT-3-Davinci-Generations": {
      "prompt_0": {
        "prompt": "Please write a why question about the above paragrpah.",
        "completions": [
          {
            "question": "Generated question 1",
            "answer": "Generated answer 1",
            "QA_quality_score": 0.5,
            "QA_ranking": 69
          },
          {
            "question": "Generated question 1",
            "answer": "Generated answer 1",
            "QA_quality_score": 0.5,
            "QA_ranking": 69
          }
        ]
      },
      "prompt_1": {
        "prompt": "Please write a what/when/where question about the above paragrpah.",
        "completions": [
          {
            "question": "Generated question 1",
            "answer": "Generated answer 1",
            "QA_quality_score": 0.5,
            "QA_ranking": 69
          },
          {
            "question": "Generated question 1",
            "answer": "Generated answer 1",
            "QA_quality_score": 0.5,
            "QA_ranking": 69
          }
        ]
      }
    }
  }
] """

In [41]:
with open('GPT-3_generations_new.json', 'w', encoding='utf-8') as f:
    json.dump(QA_data_full, f, ensure_ascii=False, indent=4) 

In [42]:
os.getcwd()

'/Users/nehasheth/Desktop/Research - AI Chatbot TA/github/data-generator/prompt_engineering'

In [48]:
s= open("/Users/nehasheth/Desktop/Research - AI Chatbot TA/github/data-generator/prompt_engineering/GPT-3_generations_new.json")
qa = json.load(s)

Paragraph level generations - approx upto 100 words each paragraph

In [ ]:

paragraph_questions, paragraph_qprompts = question_completions_with_backoff(paragraphs_list[0:500]) 

In [68]:
len(paragraph_questions)

500

In [ ]:
paragraph_answers, paragraph_QA_pairs, paragraph_QA_data_full = answer_completions_with_backoff(paragraph_questions, paragraphs_list[0:300])

In [72]:
with open('GPT-3_generations_paragraph_level.json', 'w', encoding='utf-8') as f:
    json.dump(paragraph_QA_data_full, f, ensure_ascii=False, indent=4) 